In [3]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time
import functions
from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

In [4]:
#load pcd file, filter, downsample
pcdn = o3d.io.read_point_cloud("final_cropped_ground_align.pcd")
pcdn.estimate_normals()
cl, ind = pcdn.remove_statistical_outlier(nb_neighbors=20,
                                                    std_ratio=0.8)

pcd = pcdn.select_by_index(ind)
pcd = pcd.voxel_down_sample(voxel_size=0.1)
pcd.estimate_normals()
pcd.orient_normals_consistent_tangent_plane(40)
#o3d.visualization.draw_geometries([pcd])

In [5]:

boundaries = functions.detect_boundaries(pcd)



In [6]:
geometries = functions.detect_boundary_patches(boundaries)
o3d.visualization.draw_geometries(geometries+[pcd, boundaries])
   



In [7]:
dimensions = functions.calc_mesh_area_sorted(geometries)



In [8]:
def visualize_3d_vectors(vector1, vector2):
    # Create a figure and a 3D axis
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # normalize vectors
    vector1 = vector1 / np.linalg.norm(vector1)
    vector2 = vector2 / np.linalg.norm(vector2)
    
    if functions.check_vector_similar_direction(vector1, vector2):
        pass
    else:
        pass
        #vector2 = invert_normalized_vector(vector2)
        
    
    # Extract individual coordinates from the vectors
    x1, y1, z1 = vector1
    x2, y2, z2 = vector2
    
    # Plot the vectors as lines
    ax.plot([0, x1], [0, y1], [0, z1], color='r', label='Cam orientation')
    ax.plot([0, x2], [0, y2], [0, z2], color='b', label='Plane normal orientation')
    
    # Set the limits and labels of the plot
    max_x = max(np.max(x1), np.max(x2))
    max_y = max(np.max(y1), np.max(y2))
    max_z = max(np.max(z1), np.max(z2))
    ax.set_xlim([0, 1])
    ax.set_ylim([0, 1])
    ax.set_zlim([0, 1])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    # Add a legend
    ax.legend()
    
    # Show the plot
    plt.show()

In [9]:
def create_arrow_from_vector(vector, location):
    """
    creates an arrow in form of a triangle mesh.
    Places mesh at a certain location, aligned with the input vector

    Args:
        vector (numpy.ndarray): orientation of the arrow
        location (numpy.ndarray): location of the arrow

    Returns:
        o3d.geometry.TriangleMesh: arrow with requested location and orientation
    """
    
    mesh_normal = vector
    mesh_center = location
    
    
    arrow = o3d.geometry.TriangleMesh.create_cone(0.5,1)
    
    arrow.paint_uniform_color([1,0,0])
    arrow_bb = arrow.get_oriented_bounding_box()
    arrow.compute_triangle_normals()
    arrow.compute_vertex_normals()
    arrow_normal = np.asarray(arrow.vertex_normals)[0]

    translation, rotation = functions.compute_transform(mesh_normal, arrow_normal)  
    
    arrow.rotate(rotation)
    
    arrow.translate(mesh_center, False)
    
    
    arrow_normal = np.asarray(arrow.vertex_normals)[0]
    arrow_center = arrow.get_center()
    same_dir = functions.vector_similar_direction(arrow_normal,mesh_normal)
    R_x = np.asarray([[1, 0, 0],
                       [0, -1, 0],
                       [0, 0, -1]])
    
    arrow_cp = o3d.geometry.TriangleMesh.create_cone(0.5,1)
    arrow_cp.paint_uniform_color([1,0,0])
    arrow_cp.compute_triangle_normals()
    arrow_cp.compute_vertex_normals()
    if  (same_dir):
        arrow_cp.rotate(R_x, arrow_center)
        arrow_cp.rotate(rotation)
        arrow_cp.translate(mesh_center, False)
        #return arrow_cp
        return arrow
    else:
        return arrow
    

In [10]:

def count_hits_ray_cast(scene_answer, cam_normal, dist_thresh, angle_thresh):
    """Counts the number of valid hits in a ray-casting scene.
    Takes constraints like distance to surface and surface normal into account.
    Returns percentage of valid hits.

    Args:
        scene_answer (open3d.t.geometry.RaycastingScene): input ray-casting scene. (simulated pinhole camera)
        cam_normal (numpy.ndarray): orientation of the simulated pinhole camera from ray-casting scene
        dist_thresh (tuple(float, float)): tuple containing upper and lower threshold for which a hit is counted as valid
        angle_thresh (float): ideal angle between surface normal and cam normal: 90°. angle_thresh defines maximum derivation from 90° to be still counted as valid hit

    Returns:
        float: percentage of valid hits in ray-casting scene
    """
    
    # extract surface normals and distances from ray-casting scene
    cast_normals = scene_answer['primitive_normals'].numpy()
    cast_distances = scene_answer['t_hit'].numpy()
    
    dist_lower = dist_thresh[0]
    dist_upper = dist_thresh[1]
    n_valid_hits = 0
    
    # flatten scene answers from n_pixesl x n_pixels to 1 x n_pixels²
    flattened_dist_array = cast_distances.flatten()
    n_pixels = len(flattened_dist_array)
    flattened_normal_array = cast_normals.reshape(n_pixels,3)
    

    # count valid hits: only if in distance bounds and if surface normal does not diverge more then angle_thresh degrees
    for i in range(n_pixels):
        
        if (flattened_dist_array[i] >= dist_lower) & (flattened_dist_array[i] <= dist_upper):
            pass
        else:
            continue
        
        point_normal = flattened_normal_array[i]
        angle = functions.angle_between_vectors(point_normal, cam_normal)
        
        if functions.is_perpendicular(angle, angle_thresh):
            continue
        else:
            n_valid_hits += 1
        
    
    return n_valid_hits/n_pixels

In [11]:
def rank_all_views(hole_patches, dist_thresh, angle_thresh, camera_fov=90):
    """Calculate percentage of valid hits for all potential views.
    Returns percentage of valid hits, camera position and orientation as well as ray-casting scene answers.

    Args:
        hole_patches (list[o3d.geometry.TriangleMesh]): list of triangle meshes representing the holes in a pointcloud
        dist_thresh (tuple(float, float)): tuple containing upper and lower bound for determining valid hits. valid if: lower bound <= distance hit <= upper bound
        angle_thresh (float): ideal angle between surface normal and cam normal: 90°. angle_thresh defines maximum derivation from 90° to be still counted as valid hit
        camera_fov(int): camera field of view for pinhole camera in ray-casting scene. Default: 90°

    Returns:
        list: returns percentage of valid hits, with corresponding camera position and orientation as well as ray-casting scene answers
    """
    
    # create empty ray-casting scene and variables
    scene = o3d.t.geometry.RaycastingScene()
    hole_centers = []
    hole_normals = []
    cam_positions = []
    cam_normals = []
    hit_areas = []
    scene_answers = []
    dist_threshes = [dist_thresh]*len(hole_patches)
    angle_threshes = [angle_thresh]*len(hole_patches)
    
    
    # add meshes that represent holes to ray-casting scene
    for hole in hole_patches:
        scene.add_triangles(o3d.t.geometry.TriangleMesh.from_legacy(hole))
        center = hole.get_center()
        normal = np.asarray(hole.vertex_normals)[0]
        cam_pos = functions.add_vector_to_point(center, normal, 5)
        cam_pos[2] = 0.5
        center[2] = 0.5
        hole_centers.append(center)
        hole_normals.append(normal)
        cam_positions.append(cam_pos)
        cam_normal = (center-cam_pos)
        cam_normals.append(cam_normal)
    
    
    # create rays representing virtual pinhole camera for all camera positions and orientations
    for i in range(len(hole_patches)):
        rays = o3d.t.geometry.RaycastingScene.create_rays_pinhole(
                fov_deg=camera_fov,
                center=hole_centers[i],
                eye=cam_positions[i],
                up=[0, 0, -1],
                width_px=500,
                height_px=500)
        ans = scene.cast_rays(rays)
        scene_answers.append(ans)
    
    
    # calculate percentage of valid hits for all scenes in parallel
    with Pool(mp.cpu_count()) as p:
        hit_areas = p.starmap(count_hits_ray_cast, zip(scene_answers, cam_normals, dist_threshes, angle_threshes))
  
    
    
    return list(zip(hit_areas, cam_positions, cam_normals, scene_answers))



In [12]:
     
ranked_views2 = rank_all_views(geometries, (2,10), 10 )



In [20]:
#sort views in ascending order according to area covered
#ranked_views2.sort(key=lambda x:x[0])
#ranked_views2.reverse()

ranked_views2 = functions.sort_xd_list(ranked_views2, 0, "descending")
hit_areas = [view[0] for view in ranked_views2]
best_positions = [view[1] for view in ranked_views2]
best_orientations = [view[2] for view in ranked_views2]

print(hit_areas)
print(best_positions)
print(best_orientations)
#best_pos = ranked_views2[-1][1]
#best_orientation = ranked_views2[-1][2]

#print(best_pos, best_orientation)


[0.59274, 0.405264, 0.38114, 0.379416, 0.36838, 0.357748, 0.303948, 0.29562, 0.292312, 0.284184, 0.275228, 0.259588, 0.249152, 0.238288, 0.232852, 0.232228, 0.227536, 0.224392, 0.212708, 0.190496, 0.1727, 0.165368, 0.162296, 0.149132, 0.144652, 0.144124, 0.142104, 0.138116, 0.134896, 0.125304, 0.124584, 0.121344, 0.11854, 0.117916, 0.116268, 0.112456, 0.112312, 0.101168, 0.099232, 0.090364, 0.089796, 0.088616, 0.083456, 0.07226, 0.071612, 0.067092, 0.064028, 0.06398, 0.056604, 0.049112, 0.048736, 0.045504, 0.0416, 0.041472, 0.028992, 0.021744, 0.020012, 0.014672, 0.006172, 0.0007, 0.000284, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[array([-14.73114434,   6.19587404,   0.5       ]), array([-11.03307737,  -0.41914462,   0.5       ]), array([-6.68586705,  6.00118498,  0.5       ]), array([-8.81975464,  5.0558951 ,  0.5       ]), array([-7.59471824, -2.99722096,  0.5       ]), array([-5.67830912,  6.64120847,  0.5       ]), array([1.9182121 , 8.02002782, 0.5       ]), array([1.8732424 , 7.5

In [14]:
#vec1 = np.asarray([-1,0,0])
#vec2 = np.asarray([0,1,2])
#
#_, rot_mat = compute_transform(vec1 ,vec2)
#print(rotation_matrix_to_axis_angle(rot_mat))
#print(angle_between_vectors(vec1, vec2))
#visualize_3d_vectors(vec1 ,vec2)


In [15]:
#for i in range(1,10):
#    print("Areea hit: " , ranked_views2[-i][0])
#    best_pos = ranked_views2[-i][1]
#    best_orientation = ranked_views2[-i][2]
#    best_ans = ranked_views2[-i][3]
#
#    normals = best_ans['primitive_normals'].numpy()
#    #normals = np.abs(best_ans['primitive_normals'].numpy())
#    hits = best_ans['t_hit'].numpy()
#    normals_interp = np.interp(normals, (normals.min(), normals.max()), (0, 1))
#    plt.figure(figsize=(10,6))
#    plt.imshow(normals_interp) 

In [16]:
#best_positions = []
#
#for i in range(1,10):
#    best_pos = ranked_views2[-i][1]
#    best_orientation = ranked_views2[-i][2]
#    best_ans = ranked_views2[-i][3]
#
#    
#    best_positions.append(best_pos)
#    cone = create_arrow_from_vector(best_orientation, best_pos)
#
#    #o3d.visualization.draw_geometries([pcd] + [cone]  + geometries)
#print(best_positions)

In [17]:
#np.savetxt("best_pos.csv", best_positions, delimiter="; ")